# Inteligência Artificial - Exercício Árvores

Implemente o A\* para resolver o quebra-cabeças abaixo com duas heurísticas distintas: 
    
    - h1(n) = número de pedras fora do lugar
    - h2(n) = distância total à la Manhattan (horizontal ou vertical) = somatório do deslocamento de cada peça até a posição final

Entregar o código fonte e respostas às seguintes questões (ambos no moodle):
    - Qual o tamanho do espaço de estados?
    - Para as duas heurísticas, justificar se são admissíveis e consistentes.
    - Compare as soluções encontradas com h1(n) e h2(n) em termos de:
        * solução encontrada;
        * tempo: número de nós criados em tempo de busca;
        * espaço: número máximo de nós armazenados em memória em tempo de busca.

![puzzle](res/a_estrela_puzzle.png)


As heurísticas solicitadas para resolução desse problema foram: Distância Manhattan e Peças fora do lugar. Sendo assim, definiu-se duas funções para retornar um valor referente a essas heurísticas(dados dois estados para comparação)

In [1]:
from scipy.spatial import distance
import time
MANHATTAN = 1
PIECES = 2

# funcao auxiliar para retornar posicao no jogo para o calculo da distancia manhattan
def index2d(index):
    return int(index / 3), index % 3

#calcula o somatorio da distancia manhatan de todas as peças(incluido a casa vazia)
def manhattan(matrix1, matrix2):
    dist = 0
    for i in matrix1:
        dist += distance.cityblock(index2d(matrix1.index(i)),index2d(matrix2.index(i)))
    return dist

#calcula o somatorio de peças fora do ligar(incluido a casa vazia)
def pieces(matrix1,matrix2):
    pieces = 0
    for i in matrix1:
        if matrix1.index(i) != matrix2.index(i):
            pieces += 1
    return pieces

def heuristic(matrix1, matrix2, heuristic):
    if heuristic == MANHATTAN:
        return manhattan(matrix1, matrix2)
    elif heuristic == PIECES:
        return pieces(matrix1, matrix2)

Para facilitar o algoritmo A\* vamos criar uma classe para mapear o estado do nosso problema. Todas as informações que a gente precisa soobre o estado estarão nos parâmetros.

In [2]:

class State:
    def __init__(self, configuration, actual_cost, heuristic_cost, came_from):
        self.configuration = configuration
        self.actual_cost = actual_cost
        self.heuristic_cost = heuristic_cost
        self.total_cost = self.actual_cost + self.heuristic_cost
        self.came_from = came_from
    def __cmp__(self,other):
        return (self.total_cost > other.total_cost) - (self.total_cost < other.total_cost)

No Algoritmo do agente será necessário a função que gerará os filhos do estado atual. O conceito básico dessa função será encontrar a posição da casa vazia e gerar estados mudando as peças vizinhas.

In [3]:
def generate_children(state,final_configuration,method):
    # lista com os filhos
    children = []

    # encontrando a posicao da casa vazia e as casas vizinhas
    empty_pos = state.configuration.index(0)
    above_pos = empty_pos - 3
    below_pos = empty_pos + 3
    right_pos = empty_pos + 1
    left_pos = empty_pos - 1

    # trocando com a posicao ACIMA caso existir
    if above_pos >= 0:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[above_pos]
        new_conf[above_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ABAIXO caso existir
    if below_pos < len(state.configuration):
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[below_pos]
        new_conf[below_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao DIREITA
    if right_pos < len(state.configuration):
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[right_pos]
        new_conf[right_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ESQUERDA caso existir
    if left_pos >= 0:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[left_pos]
        new_conf[left_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    return children

Também será necessário criar funções auxilliares para controlar a fronteira e imprimir os estados formatados para que pareçam-se com o jogo

In [4]:
def add_to_border(state_list, border):
    already_on_border = False
    for state in state_list:
        for border_state in border:
            if state.configuration == border_state.configuration:
                already_on_border = True
                if state.actual_cost < border_state.actual_cost:
                    border[border.index(border_state)] = state
        if not already_on_border:
            border.append(state)

def print_as_table(configuration):
    l1=""
    l2=""
    l3=""
    for i in configuration:
       if configuration.index(i) < 3:
           l1 += str(i)
       elif 3 <= configuration.index(i) < 6:
           l2 += str(i)
       else:
           l3 += str(i)
    print(l1)
    print(l2)
    print(l3)
    print("---")

Com a estrutura básica criada, agora é necessário modelar o agente que irá fazer a busca A\*. A ideia básica do agente é:
1. adiciona estado na fronteira
2. enquanto fronteira != de vazio
    - seleciona o melhor da fronteira removendo ele da fronteira e adicionando aos "estados visitados"
    - se for estado final, adiciona aos estados e percorre o caminho came_from --> fim
    - se nao for, gera os estados filhos adicionando na fronteira
    - volta 2

In [5]:
def algorithm(initial_state, final_state, heuristic_method):
    start_time = time.time()
    generation = 0
    method = heuristic_method

    # fronteira
    border = []

    # estados visitados
    visited = []

    # representando os estados
    si = initial_state
    sf = final_state

    found_solution = False

    # Criando o estado inicial com todos os valores
    start_state = State(si, 0, heuristic(si, sf, method), None)

    # adicionando o estado inicial à fronteira
    border.append(start_state)

    # enquanto a fronteira nao estiver vazia
    while len(border) > 0:
        generation += 1

        # ordenando para poder obter o melhor a cada iteração
        border.sort(key=lambda state: state.total_cost)

        # selecionando o melhor e colocando na lista dos nós visitados
        actual_state = border[0]
        visited.append(actual_state)

        # removendo o nó visitado
        del (border[0])

        # verificando se o estado já é o final
        if actual_state.configuration == sf:
            found_solution = True
            break
        else:
            # gerando os estados filhos e adicionado à fronteira
            add_to_border(generate_children(actual_state, sf, method), border)

    if found_solution:
        state = visited.pop()
        result = []
        while state is not start_state:
            result.append(state)
            state = state.came_from

        #incuindo o no inicial ao resultado
        result.append(start_state)
        #inverntendo a ordem para imprimir o passo correto
        result.reverse()

        print('PASSOS PARA A SOLUCAO' + '\n')
        for r in result:
            print_as_table(r.configuration)
    else:
        print('Nao encontrou solucao')
    end_time = time.time()
    print('Tempo total: ' + str(end_time - start_time) +str(' segundos'))
    print('Geracoes: ' + str(generation))

Aqui rodamos o algoritmo com a heruristica de distancia Manhattan

In [6]:
initial_state = [7, 2, 4,
                     5, 0, 6,
                     8, 3, 1]

final_state = [0, 1, 2,
               3, 4, 5,
               6, 7, 8]

heuristic_method = MANHATTAN

algorithm(initial_state, final_state, heuristic_method)

PASSOS PARA A SOLUCAO

724
506
831
---
724
560
831
---
724
568
031
---
724
568
301
---
724
568
310
---
724
560
318
---
724
563
018
---
724
063
518
---
024
763
518
---
204
763
518
---
240
763
518
---
243
760
518
---
243
768
510
---
243
760
518
---
243
765
018
---
243
765
108
---
243
765
018
---
243
065
718
---
243
605
718
---
243
615
708
---
243
615
078
---
243
015
678
---
243
105
678
---
203
145
678
---
023
145
678
---
123
045
678
---
120
345
678
---
102
345
678
---
012
345
678
---
Tempo total: 174.18610072135925 segundos
Geracoes: 31093


Aqui rodamos o algortimo com a heuristica de peças fora do lugar

In [7]:
initial_state = [7, 2, 4,
                     5, 0, 6,
                     8, 3, 1]

final_state = [0, 1, 2,
               3, 4, 5,
               6, 7, 8]

heuristic_method = PIECES

algorithm(initial_state, final_state, heuristic_method)

PASSOS PARA A SOLUCAO

724
506
831
---
724
560
831
---
724
568
031
---
724
568
301
---
724
568
310
---
724
560
318
---
724
563
018
---
724
063
518
---
024
763
518
---
204
763
518
---
240
763
518
---
247
063
518
---
247
603
518
---
247
630
518
---
240
637
518
---
204
637
518
---
234
607
518
---
234
670
518
---
234
675
018
---
234
675
108
---
234
605
178
---
234
650
178
---
234
651
078
---
234
051
678
---
034
251
678
---
304
251
678
---
340
251
678
---
342
051
678
---
340
251
678
---
341
250
678
---
341
205
678
---
301
245
678
---
310
245
678
---
312
045
678
---
012
345
678
---
Tempo total: 1437.4984683990479 segundos
Geracoes: 102316
